# XGBoost solution

**_______________________________________________________________________________________________________________________________**

**Загрузим все необходимые зависимости и сущности, модули:**

In [1]:
%run ../preset.py

---------------------------------------------------------------------------------------------------------------------------

**Загрузим готовые данные:**

In [2]:
df = pd.read_parquet(feature_data_url)

___________________________________________________________________________________________________________________________

**Обзор dataframe-a:**

In [3]:
df.shape

(5588845, 91)

In [4]:
df.columns

Index(['actual_consumption', 'time_dummie', 'year', 'month', 'day_of_month',
       'hour', 'day_of_week', 'day_of_year', 'week_of_year', 'quarter',
       'holiday', 'lag_hour', 'lag_day', 'lag_week', 'lag_month', 'lag_year',
       'Алтайский край', 'Амурская область', 'Архангельская область',
       'Астраханская область', 'Белгородская область', 'Брянская область',
       'Владимирская область', 'Волгоградская область', 'Вологодская область',
       'Воронежская область', 'Еврейская автономная область',
       'Забайкальский край', 'Западный энергорайон Якутии',
       'Ивановская область', 'Иркутская область',
       'Кабардино-Балкарская Республика', 'Калининградская область',
       'Калужская область', 'Карачаево-Черкесская Республика',
       'Кемеровская область - Кузбасс', 'Кировская область',
       'Костромская область', 'Краснодарский край', 'Красноярский край',
       'Курганская область', 'Курская область', 'Ленинградская область',
       'Липецкая область', 'Московская

In [5]:
df.head(3)

,actual_consumption,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2012-02-20 00:00:00,1302.0,8783,2012,2,20,0,1,51,8,3,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,1091.0,8783,2012,2,20,0,1,51,8,3,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,946.0,8783,2012,2,20,0,1,51,8,3,...,0,0,0,0,0,0,0,0,0,0


**Рассмотрим информацию о dataframe-е:**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5588845 entries, 2012-02-20 00:00:00 to 2024-07-17 00:00:00
Data columns (total 91 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   actual_consumption                 float64
 1   time_dummie                        int64  
 2   year                               int64  
 3   month                              int64  
 4   day_of_month                       int64  
 5   hour                               int64  
 6   day_of_week                        int64  
 7   day_of_year                        int64  
 8   week_of_year                       int64  
 9   quarter                            int64  
 10  holiday                            int64  
 11  lag_hour                           float64
 12  lag_day                            float64
 13  lag_week                           float64
 14  lag_month                          float64
 15  lag_year                           float6

___________________________________________________________________________________________________________________________

**Отберём целевой признак:**

In [7]:
y = df["actual_consumption"].copy()

**Рассмотрим целевой признак:**

In [8]:
y

index
2012-02-20 00:00:00    1302.0
2012-02-20 00:00:00    1091.0
2012-02-20 00:00:00     946.0
2012-02-20 00:00:00     615.0
2012-02-20 00:00:00    1679.0
                        ...  
2024-07-17 00:00:00    3854.0
2024-07-17 00:00:00     412.0
2024-07-17 00:00:00     486.0
2024-07-17 00:00:00     358.0
2024-07-17 00:00:00     766.0
Name: actual_consumption, Length: 5588845, dtype: float64

___________________________________________________________________________________________________________________________

**Отберём переменные-регрессоры:**

In [9]:
x = df.drop(columns=["actual_consumption"])

**Рассмотрим отобранные признаки:**

In [10]:
x.head(3)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0


___________________________________________________________________________________________________________________________

**Разобъём данные на три выборки (тренировочную, тестовую, валидационную) без перемешивания:**

In [11]:
x_train, x_other, y_train, y_other = train_test_split(
                                                      x,  # Значения признаков
                                                      y,  # Значения целевой переменной
                                                      test_size=0.3,  # Размер тестовой выборки
                                                      shuffle=False  # Отключаем перемешивание данных
                                                     )

In [12]:
x_test, x_val, y_test, y_val = train_test_split(
                                                x_other,  # Значения признаков
                                                y_other,  # Значения целевой переменной
                                                test_size=0.25,  # Размер тестовой выборки
                                                shuffle=False  # Отключаем перемешивание данных
                                               )

**Рассмотрим полученные выборки:**

In [13]:
x_train.head(3)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
x_test.head(3)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
x_val.head(3)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2023-08-24 07:00:00,107980,2023,8,24,7,4,236,34,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-08-24 07:00:00,107978,2023,8,24,7,4,236,34,3,0,...,0,0,0,0,0,0,0,0,0,0
2023-08-24 07:00:00,107976,2023,8,24,7,4,236,34,3,0,...,0,0,0,0,0,0,0,0,0,0


___________________________________________________________________________________________________________________________

**Ознакомимся с моделью машинного обучения:**

In [16]:
help(XGBRegressor)

Help on class XGBRegressor in module xgboost.sklearn:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  See :doc:`/python/sklearn_estimator` for more information.
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : Optional[int]
 |          Number of gradient boosted trees.  Equivalent to number of boosting
 |          rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
 |          d

___________________________________________________________________________________________________________________________

**Создадим модель машинного обучения:**

In [17]:
main_model = xgb.XGBRegressor(n_estimators=2500, learning_rate=0.075)

___________________________________________________________________________________________________________________________

**Рассмотрим параметры сетки:**

In [18]:
# help(GridSearchCV)

**Установим параметры сетки:**

In [19]:
# parameters = {
# #               "max_depth":[2, 3],
# #               "max_leaves": [4, 5],
# #               "learning_rate":[0.05, 0.025],
#               "n_estimators": [1000]
#              }

**Создадим сетку:**

In [20]:
# GS_model = GridSearchCV(
#                         main_model,
#                         param_grid=parameters,
#                         cv=4,
#                         scoring="neg_mean_squared_error"
#                        )

**Найдём лучшие параметры при помощи сетки:**

In [49]:
x_train

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
index,,,,,,,,,,,,,,,,,,,,,
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2012-02-20 00:00:00,8783,2012,2,20,0,1,51,8,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-15 07:00:00,84518,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84522,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-15 07:00:00,84522,2020,12,15,7,2,350,51,3,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
main_model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.075, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

**Рассмотрим лучшие параметры подобранные с помощью сетки:**

In [22]:
# print(GS_model.best_params_)

___________________________________________________________________________________________________________________________

**Спрогнозируем данные на валидационной выборке:**

In [23]:
y_pred = main_model.predict(x_test)

**Оценим полученный результат:**

In [50]:
y_test

index
2020-12-15 07:00:00      92.0
2020-12-15 07:00:00    1009.0
2020-12-15 07:00:00    1065.0
2020-12-15 07:00:00    1147.0
2020-12-15 07:00:00     440.0
                        ...  
2023-08-24 07:00:00    1068.0
2023-08-24 07:00:00    1210.0
2023-08-24 07:00:00    1670.0
2023-08-24 07:00:00     290.0
2023-08-24 07:00:00     488.0
Name: actual_consumption, Length: 1257490, dtype: float64

In [24]:
check_res(y_pred, y_test)

Значения метрик:
                              MSE - 11497.409188199184
                              MAE - 37.47860209377637
                              MAPE - 0.023173561071007247


In [25]:
res_x = pd.concat([x_train, x_test])

In [26]:
res_y = pd.concat([y_train, y_test])

In [27]:
main_model.fit(res_x, res_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.075, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [28]:
y_new_pred = main_model.predict(x_val)

In [29]:
check_res(y_new_pred, y_val)

Значения метрик:
                              MSE - 19634.262516316892
                              MAE - 41.57008331496423
                              MAPE - 0.022570440485266047


**Создание pipeline-а:**

In [30]:
test_df = pd.read_csv("../../parsed_data/new.csv")

In [31]:
test_df.head()

,Unnamed: 0,$id,Id,Name,ParentPowerSystemId,PowerSystemId,SubjectId,Disabled,SvgPaths,Top,...,IBR_PlannedConsumption,IBR_ActualConsumption,IBR_PlannedGeneration,IBR_ActualGeneration,IBR_AveragePrice,VSVGO_AveragePrice,VSVGO_Consumption,Selected,InfoCoordinates,Datetime
0,0,21,530000_14,Белгородская область,530000,530000,14,False,['M0 0 M24 237 L25 236 L24 235 L23 233 L22 234...,233,...,1 607 МВт*ч,1 653 МВт*ч,53 МВт*ч,54 МВт*ч,934 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
1,1,22,530000_15,Брянская область,530000,530000,15,False,['M0 0 M53 154 L52 153 L53 152 L53 152 L53 151...,147,...,380 МВт*ч,378 МВт*ч,-,-,928 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
2,2,23,530000_17,Владимирская область,530000,530000,17,False,['M0 0 M169 141 L167 141 L165 142 L164 140 L16...,123,...,701 МВт*ч,707 МВт*ч,126 МВт*ч,126 МВт*ч,976 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
3,3,24,530000_19,Вологодская область,530000,530000,19,False,['M0 0 M256 54 L252 54 L247 50 L248 48 L242 46...,0,...,1 404 МВт*ч,1 409 МВт*ч,1 144 МВт*ч,997 МВт*ч,928 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
4,4,25,530000_20,Воронежская область,530000,530000,20,False,['M0 0 M103 253 L102 252 L101 251 L100 250 L99...,234,...,1 225 МВт*ч,1 246 МВт*ч,3 492 МВт*ч,3 514 МВт*ч,938 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00


In [32]:
def create_time_step_features(df):
    df["time_dummie"] = create_time_dummie_feature(df)
    df["year"] = df["datetime"].apply(get_year)
    df["month"] = df["datetime"].apply(get_month)
    df["day_of_month"] = df["datetime"].apply(get_day_month)
    df["hour"] = df["datetime"].apply(get_hour)
    df["day_of_week"] = df["datetime"].apply(get_day_week)
    df["day_of_year"] = df["datetime"].apply(get_day_year)
    df["week_of_year"] = df["datetime"].apply(get_week_year)
    df["quarter"] = df["datetime"].apply(get_quarter)
    df["holiday"] = df["datetime"].apply(is_holiday)
    
    return df

In [33]:
def get_real_values(df, dummies):
    """
    Восстанавливает оригинальные значения из закодированных дамми-переменных.
    
    Параметры:
    df (pd.DataFrame) - исходный DataFrame
    dummies (list) - список названий дамми-переменных
    
    Возвращает:
    pd.DataFrame - DataFrame с восстановленными исходными значениями
    """
    # Создаем копию DataFrame, чтобы не модифицировать исходный
    df_real = df.copy()
    
    # Находим уникальные значения для каждой дамми-переменной
    unique_values = {dummy: df[dummy].unique() for dummy in dummies}
    
    # Для каждой дамми-переменной
    for dummy, values in unique_values.items():
        # Находим индексы, где данная дамми-переменная = 1
        for value in values:
            indexes = df[dummy] == value
            
            # Восстанавливаем исходное значение
            df_real.loc[indexes, dummy] = dummy[1]
        
        # Удаляем дамми-переменную
        df_real = df_real.drop(dummy, axis=1)
    
    return df_real

In [34]:
def find_index_with_one(row):
    for i, val in enumerate(row):
        if val == 1:
            return i
    return None

In [35]:
def decoding(encoding_columns):
    zxc = encoding_columns.apply(find_index_with_one, axis=1)
    tmp = Subj_req[0]
    
    return [tmp[i] for i in zxc]

In [36]:
def prepare_df(df_name: str, data_format: str) -> pd.DataFrame:
    # Загружаем всё:
    base_df = pd.read_parquet("../../prepare_data/feature_data.gzip")
    test_df = pd.read_csv(f"../../parsed_data/{df_name}.{data_format}")
    
    test_df.rename(columns={
                   "Name": "subject_name",
                   "Datetime": "datetime",
                   "IBR_ActualConsumption": "actual_consumption"
                  }, inplace=True)
    
    test_df.sort_values(by=["subject_name", "datetime"], inplace=True)
    
    test_df["actual_consumption"] = test_df["actual_consumption"].apply(to_int)
    test_df.drop_duplicates(inplace=True)
    test_df.dropna(inplace=True)

    test_df = test_df[["subject_name", "datetime", "actual_consumption"]].copy()
    
    n = test_df.shape[0]
    
    # Тут всё:
    
    test_df = create_time_step_features(test_df)
    zxc = pd.get_dummies(test_df["subject_name"], columns=Subj_req)
    test_df = pd.concat([test_df, zxc], axis=1)
#     test_df.drop(columns=["datetime"], inplace=True)
    rows = test_df.columns
    
    base_df["subject_name"] = decoding(base_df[Subj_req[0]])
    base_df["datetime"] = base_df.index
    new_df = pd.concat([base_df[rows], test_df], axis=0, ignore_index=True)
  
    

    new_df["lag_hour"] = create_lag_feature(new_df, hour)
    new_df["lag_day"] = create_lag_feature(new_df, day)
    new_df["lag_week"] = create_lag_feature(new_df, week)
    new_df["lag_month"] = create_lag_feature(new_df, month)
    new_df["lag_year"] = create_lag_feature(new_df, year)

    new_df = new_df[new_df["day_of_week"] < 6]
     
    new_df.sort_values(by=["datetime", "subject_name"], inplace=True)
    new_df.dropna(inplace=True)
    new_df.drop(columns=["datetime", "subject_name"], inplace=True)
    
    return new_df.iloc[-n:]

In [37]:
res = prepare_df("new", "csv")

In [38]:
test_df = pd.read_csv(f"../../parsed_data/new.csv")

In [39]:
test_df

,Unnamed: 0,$id,Id,Name,ParentPowerSystemId,PowerSystemId,SubjectId,Disabled,SvgPaths,Top,...,IBR_PlannedConsumption,IBR_ActualConsumption,IBR_PlannedGeneration,IBR_ActualGeneration,IBR_AveragePrice,VSVGO_AveragePrice,VSVGO_Consumption,Selected,InfoCoordinates,Datetime
0,0,21,530000_14,Белгородская область,530000,530000,14,False,['M0 0 M24 237 L25 236 L24 235 L23 233 L22 234...,233,...,1 607 МВт*ч,1 653 МВт*ч,53 МВт*ч,54 МВт*ч,934 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
1,1,22,530000_15,Брянская область,530000,530000,15,False,['M0 0 M53 154 L52 153 L53 152 L53 152 L53 151...,147,...,380 МВт*ч,378 МВт*ч,-,-,928 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
2,2,23,530000_17,Владимирская область,530000,530000,17,False,['M0 0 M169 141 L167 141 L165 142 L164 140 L16...,123,...,701 МВт*ч,707 МВт*ч,126 МВт*ч,126 МВт*ч,976 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
3,3,24,530000_19,Вологодская область,530000,530000,19,False,['M0 0 M256 54 L252 54 L247 50 L248 48 L242 46...,0,...,1 404 МВт*ч,1 409 МВт*ч,1 144 МВт*ч,997 МВт*ч,928 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
4,4,25,530000_20,Воронежская область,530000,530000,20,False,['M0 0 M103 253 L102 252 L101 251 L100 250 L99...,234,...,1 225 МВт*ч,1 246 МВт*ч,3 492 МВт*ч,3 514 МВт*ч,938 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12670,12670,15,840000_47,Мурманская область,840000,840000,47,False,['M0 0 M222 17 L219 16 L218 16 L218 17 L216 17...,0,...,1 019 МВт*ч,1 034 МВт*ч,1 826 МВт*ч,1 846 МВт*ч,-,-,-,False,"{'X': 0, 'Y': 0}",2024-08-08 00:00:00
12671,12671,16,840000_49,Новгородская область,840000,840000,49,False,['M0 0 M100 134 L99 135 L98 135 L97 134 L97 13...,129,...,446 МВт*ч,445 МВт*ч,248 МВт*ч,261 МВт*ч,1 115 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-08 00:00:00
12672,12672,17,840000_58,Псковская область,840000,840000,58,False,['M0 0 M93 117 L90 117 L90 117 L89 117 L89 118...,116,...,201 МВт*ч,204 МВт*ч,-,-,1 144 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-08 00:00:00
12673,12673,18,840000_86,Республика Карелия,840000,840000,86,False,['M0 0 M141 89 L141 90 L141 90 L140 90 L140 90...,31,...,857 МВт*ч,864 МВт*ч,510 МВт*ч,514 МВт*ч,797 руб./МВт*ч,-,-,False,"{'X': 0, 'Y': 0}",2024-08-08 00:00:00


In [40]:
res

,actual_consumption,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,...,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область,lag_hour,lag_day,lag_week,lag_month,lag_year
5585245,921.0,115757,2024,7,15,1,1,197,29,3,...,0,0,0,0,0,866.0,909.0,844.0,869.0,1577.0
5585246,928.0,115755,2024,7,15,1,1,197,29,3,...,0,0,0,0,0,885.0,1002.0,1028.0,1008.0,1462.0
5585247,595.0,115757,2024,7,15,1,1,197,29,3,...,0,0,0,0,0,620.0,602.0,573.0,642.0,966.0
5585248,447.0,115750,2024,7,15,1,1,197,29,3,...,0,0,0,0,0,459.0,492.0,436.0,324.0,625.0
5585249,1478.0,115755,2024,7,15,1,1,197,29,3,...,0,0,0,0,0,1588.0,1647.0,1552.0,1784.0,2129.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5600842,3742.0,169,2024,8,8,0,4,221,32,3,...,1,0,0,0,0,3842.0,3666.0,3700.0,3730.0,4671.0
5601011,350.0,169,2024,8,8,0,4,221,32,3,...,0,1,0,0,0,381.0,408.0,346.0,305.0,451.0
5601180,471.0,169,2024,8,8,0,4,221,32,3,...,0,0,1,0,0,487.0,476.0,456.0,426.0,778.0
5601349,359.0,169,2024,8,8,0,4,221,32,3,...,0,0,0,1,0,361.0,342.0,340.0,372.0,393.0


In [41]:
y = res["actual_consumption"]

In [42]:
x = res.drop(columns=["actual_consumption"])

In [44]:
# x.drop(columns=["datetime", "subject_name"], inplace=True)

In [45]:
x = x[res_x.columns]

In [47]:
y_last = main_model.predict(x)

In [48]:
check_res(y_last, y)

Значения метрик:
                              MSE - 12765.845036238616
                              MAE - 38.546512945378936
                              MAPE - 0.026370953894278463
